# Load Packages

In [1]:
import os
import cv2
import math
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from glob import glob
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import shutil
#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

# File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"
tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")
tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
#https://stackoverflow.com/questions/5953373/how-to-split-image-into-multiple-pieces-in-python
correct_directory = os.path.join(parent_directory,"temp/rechip")
incorrectly_chipped_images_path = os.path.join(parent_directory,"temp/incorrectly_chipped_images")
remaining_chips_path =  os.path.join(parent_directory,"temp/remaining_chips")
remaining_sets_path =  os.path.join(parent_directory,"temp/remaining_sets")



# If Necessary: Download Additional Tiles 

If necessary, download additional tiles
Note: Reform into function, to call above.

## Set 3

In [35]:
unverified_set3_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set3")
unverified_set3_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set3_subfolders_path))


## Get a list of the tiles in the subfolder
tiles = []
for directory in tqdm.tqdm(unverified_set3_subfolders_paths):
    #get all the image and xml paths in directory of annotated images
    #print(directory)
    fc.remove_thumbs(directory[0])
    image_paths = glob(directory[0] + "/*.jpg", recursive = True)
    xml_paths = glob(directory[1] + "/*.xml", recursive = True)
    #print(len(image_paths),len(xml_paths))

    #identify tiles in each folder
    for image in image_paths:
        image_name = os.path.splitext(os.path.basename(image))[0]
        tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0]
        tiles.append(tile_name)
tiles = np.unique(tiles)

## Get a list of the tiles downloaded (without the extension)
tiles_downloaded =  os.listdir(tiles_complete_dataset_path)
tiles_downloaded_wo_ext = []
for tile_downloaded in tiles_downloaded:
    tiles_downloaded_wo_ext.append(os.path.splitext(tile_downloaded)[0])
print(len(tiles_downloaded_wo_ext))

## Get a list of the the tiles that need to be downloaded
tiles_to_download = list(set(tiles.tolist()) - set(tiles_downloaded_wo_ext)) 

## Download Tiles
tile_names = []
tile_urls = []
file_names = []
for tile in tiles_to_download:   
    ### download the tiles if they are not in the tiles folder 
    #check if the tile name is contained in the string of complete arrays
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile in string]          
    if len(tile_name) == 1: #A single tile name # get tile url from the first (only) entry
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] 
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif len(np.unique(tile_name)) > 1: # Multiple (different tiles) possibly the same tiles in different states, possible different years
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1]# get tile url
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif (len(tile_name) > 1): #Multiple different tile names that are the same, probably different naip storage locations
        # get tile url from the second entry 
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[1]][1][1] 
        tile_names.append(tile_name[1])
        tile_urls.append(tile_url)

    #get file name
    file_name = tile_name[0]
    if tile_name[0].count("_") > 5:
        tile_name = tile_name[0].rsplit("_",1)[0]
        file_name = tile_name + ".tif"
    print(file_name)
    ### Download tile
    file_names.append(ap.download_url(tile_url, tiles_complete_dataset_path,
                                                 destination_filename = file_name,       
                                                         progress_updater=ap.DownloadProgressBar()))

m_3411762_sw_11_060_20180822.tif


100% (408585536 of 408585536) |##########| Elapsed Time: 0:00:42 Time:  0:00:42


...done, 408585536 bytes.
m_3009612_nw_14_060_20181102.tif


100% (438599822 of 438599822) |##########| Elapsed Time: 0:00:43 Time:  0:00:43


...done, 438599822 bytes.
m_3809031_nw_15_060_20181121.tif


...

...done, 508105636 bytes.
m_3508057_ne_17_060_20181107.tif


100% (445281987 of 445281987) |##########| Elapsed Time: 0:00:41 Time:  0:00:41


...done, 445281987 bytes.
m_3909451_se_15_060_20180923.tif


100% (421702554 of 421702554) |##########| Elapsed Time: 0:00:36 Time:  0:00:36


...done, 421702554 bytes.


## Set 4

In [42]:
## Get a list of the tiles in the subfolder
unverified_set4_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set4")
unverified_set4_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set4_subfolders_path))

tiles = []
for directory in tqdm.tqdm(unverified_set4_subfolders_paths):
    #get all the image and xml paths in directory of annotated images
    #print(directory)
    fc.remove_thumbs(directory[0])
    image_paths = glob(directory[0] + "/*.jpg", recursive = True)
    xml_paths = glob(directory[1] + "/*.xml", recursive = True)
    #print(len(image_paths),len(xml_paths))

    #identify tiles in each folder
    for image in image_paths:
        image_name = os.path.splitext(os.path.basename(image))[0]
        tile_name = image_name.split("_",4)[-1].rsplit("_",1)[0]
        tiles.append(tile_name)
tiles = np.unique(tiles)

## Get a list of the tiles downloaded (without the extension)
tiles_downloaded =  os.listdir(tiles_complete_dataset_path)
tiles_downloaded_wo_ext = []
for tile_downloaded in tiles_downloaded:
    tiles_downloaded_wo_ext.append(os.path.splitext(tile_downloaded)[0])
print("number of tiles downloaded", len(tiles_downloaded_wo_ext))

## Get a list of the the tiles that need to be downloaded
tiles_to_download = list(set(tiles.tolist()) - set(tiles_downloaded_wo_ext)) 

## Download Tiles
tile_names = []
tile_urls = []
file_names = []
print("number of tiles that need to be downloaded", len(tiles_to_download))
for tile in tiles_to_download:   
    ### download the tiles if they are not in the tiles folder 
    #check if the tile name is contained in the string of complete arrays
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile in string]          
    if len(tile_name) == 1: #A single tile name # get tile url from the first (only) entry
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] 
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif len(np.unique(tile_name)) > 1: # Multiple (different tiles) possibly the same tiles in different states, possible different years
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1]# get tile url
        tile_names.append(tile_name[0])
        tile_urls.append(tile_url)
    elif (len(tile_name) > 1): #Multiple different tile names that are the same, probably different naip storage locations
        # get tile url from the second entry 
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[1]][1][1] 
        tile_names.append(tile_name[1])
        tile_urls.append(tile_url)

    #get file name
    file_name = tile_name[0]
    if tile_name[0].count("_") > 5:
        tile_name = tile_name[0].rsplit("_",1)[0]
        file_name = tile_name + ".tif"
    print(file_name)
    ### Download tile
    file_names.append(ap.download_url(tile_url, tiles_complete_dataset_path,
                                                 destination_filename = file_name,       
                                                         progress_updater=ap.DownloadProgressBar()))

100%|██████████| 14/14 [00:00<00:00, 36.00it/s]


2021
118
m_4208355_sw_17_060_20180707.tif


100% (512753778 of 512753778) |##########| Elapsed Time: 0:00:55 Time:  0:00:55


...done, 512753778 bytes.
m_3110238_sw_13_060_20180924.tif


100% (495216713 of 495216713) |##########| Elapsed Time: 0:00:42 Time:  0:00:42


...

...done, 381201842 bytes.
m_4408523_nw_16_060_20180912.tif


100% (513328615 of 513328615) |##########| Elapsed Time: 0:01:07 Time:  0:01:07


...done, 513328615 bytes.
m_3908004_ne_17_060_20180701.tif


100% (448611597 of 448611597) |##########| Elapsed Time: 0:01:09 Time:  0:01:09


...done, 448611597 bytes.


# Identify Remaining Images, and Rename Labeled Images/XMLs that match correct Images

## Unverified sets

### Set 1

In [3]:
unverified_set1_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set1")
unverified_set1_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set1_subfolders_path))
fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set1_subfolders_paths[32:], correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/8 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Qianyu/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Qianyu/chips_positive_xml']
24 22
remaining images 62692
24 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 22
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
25 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 62692
26 23
remaining images 6269

 12%|█▎        | 1/8 [24:43<2:53:05, 1483.59s/it]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images1/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images1/chips_positive_xml']
25 17
remaining images 62928
25 17
remaining images 63064
25 17
remaining images 63200
25 17
remaining images 63200
25 18
remaining images 63200
25 17
remaining images 63336
25 18
remaining images 63336
25 17
remaining images 63472
25 17
remaining images 63608
25 17
remaining images 63744
25 17
remaining images 63880
25 18
remaining images 64006
25 18
remaining images 64132
25 18
remaining images 64258
25 18
remaining images 64384
25 17
remaining images 64520
25 17
remaining images 64656
25 17
remaining images 64792
25 17
remaining images 64928
25 17
remaining images 65064
25 18
remaining images 65190
25 18
remaining images 65316
25 18
remaining images 65442
25 18
re

 25%|██▌       | 2/8 [1:03:54<3:19:24, 1994.05s/it]

...

 75%|███████▌  | 6/8 [2:45:38<46:43, 1401.77s/it]  

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images6/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images6/chips_positive_xml']
25 20
remaining images 81868
25 20
remaining images 81868
25 20
remaining images 81968
25 20
remaining images 81968
25 19
remaining images 81968
25 19
remaining images 82082
25 19
remaining images 82196
25 19
remaining images 82196
25 19
remaining images 82196
25 19
remaining images 82310
25 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82310
24 19
remaining images 82405
24 19
remaining images 82500
25 19
remaining images 82614
25 20
remaining images 82714
25 20
remaining images 82814
25 20
remaining images 82914
25 20
re

 88%|████████▊ | 7/8 [2:59:01<20:06, 1206.13s/it]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images7/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Sunny/student_reviewed_images7/chips_positive_xml']
26 20
remaining images 83134
25 20
remaining images 83234
25 20
remaining images 83334
25 21
remaining images 83418
26 20
remaining images 83538
25 20
remaining images 83638
26 21
remaining images 83743
26 21
remaining images 83848
25 20
remaining images 83948
25 20
remaining images 83948


100%|██████████| 8/8 [3:05:39<00:00, 1392.48s/it]


### Set 2

In [4]:
unverified_set2_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set2")
unverified_set2_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set2_subfolders_path))

fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set2_subfolders_paths[45:], correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/11 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images12_Sunny/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images12_Sunny/chips_positive_xml']


  9%|▉         | 1/11 [04:09<41:30, 249.09s/it]

remaining images 133271
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images13_Sunny/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Sunny/student_reviewed_images13_Sunny/chips_positive_xml']


 18%|█▊        | 2/11 [10:56<51:18, 342.05s/it]

...

remaining images 141837
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images5_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images5_Tang/chips_positive_xml']


 91%|█████████ | 10/11 [57:54<05:47, 347.61s/it]

remaining images 142804
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images6_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set2/Tang/student_reviewed_images6_Tang/chips_positive_xml']


100%|██████████| 11/11 [1:03:15<00:00, 345.01s/it]

remaining images 143605


### Set 3

In [36]:
unverified_set3_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set3")
unverified_set3_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set3_subfolders_path))
fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set3_subfolders_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/31 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/student_reviewed_images4_Alvarez/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/student_reviewed_images4_Alvarez/chips_positive_xml']


  3%|▎         | 1/31 [08:06<4:03:24, 486.80s/it]

remaining images 144679
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/unverified_images_not_reviewed_by_student5_Alvarez/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Alvarez/unverified_images_not_reviewed_by_student5_Alvarez/chips_positive_xml']


  6%|▋         | 2/31 [14:18<3:22:25, 418.81s/it]

...

remaining images 180235
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images7_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images7_Tang/chips_positive_xml']


 97%|█████████▋| 30/31 [3:23:30<05:06, 306.52s/it]

remaining images 181082
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images8_Tang/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set3/Tang/student_reviewed_images8_Tang/chips_positive_xml']


100%|██████████| 31/31 [3:28:27<00:00, 403.46s/it]

remaining images 181987


### Set 4

In [43]:
unverified_set4_subfolders_path = os.path.join(parent_directory,"unverified_images\student_reviewed_unverified_images_set4")
unverified_set4_subfolders_paths = ap.img_path_anno_path(ap.list_of_sub_directories(unverified_set4_subfolders_path))

fc.img_anno_paths_to_corrected_names_for_labeled_images_and_remaining_images(unverified_set4_subfolders_paths, correct_directory, incorrectly_chipped_images_path, 
                                                                              remaining_chips_path, tiles_complete_dataset_path)

  0%|          | 0/14 [00:00<?, ?it/s]

['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images7_Feinberg/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images7_Feinberg/chips_positive_xml']


  7%|▋         | 1/14 [10:27<2:15:52, 627.11s/it]

remaining images 183267
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images8_Feinberg/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Feinberg/student_reviewed_images8_Feinberg/chips_positive_xml']


 14%|█▍        | 2/14 [24:29<2:30:41, 753.47s/it]

...

remaining images 192403
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student15_Poonacha/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student15_Poonacha/chips_positive_xml']


 93%|█████████▎| 13/14 [1:24:40<05:52, 352.38s/it]

remaining images 193292
['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student16_Poonacha/chips_positive'
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set4/Poonacha/unverified_images_not_reviewed_by_student16_Poonacha/chips_positive_xml']


100%|██████████| 14/14 [1:28:37<00:00, 379.80s/it]

remaining images 193991


## Verified Sets,
Once all images of interest have been labeled, 

In [ ]:
# Create 